In [1]:
# from pdfreader import SimplePDFViewer
import pandas as pd
import numpy as np
from natsort import index_natsorted
import os
import textract
import re

# Bulk analysis

This code iterates through a folder of PDFs (Deutsche Bahn annual accounting reports), and counts the appearances of particular terms in the documents.

In [2]:
# Code applied with the help of:
# https://www.geeksforgeeks.org/how-to-read-multiple-text-files-from-folder-in-python/
# https://stackoverflow.com/questions/50317483/count-the-total-number-of-words-in-a-pdf-document-using-python

# Change the directory
path = "../Dokumente/Publikationen DB/pdfs" # Select path with pdfs
os.chdir(path)

Below a dynamic script iterates through a folder with pdf files. The `regex` functions were tested in another cell below. The larger set of code here closely follows the logic of the files collected: reports from the Bahn. As a result, early reports are excluded, due to their poor quality.


In [22]:
master_df = pd.DataFrame(columns=['Datei', 'Anzahl_Worte', 'Verkehrswende', 'Digital','Nachhaltig',
                                  'Ökologie', 'Energie', 'Klima', 'Reform', 'Obsoleszenz', 'Verspätung', 'Wettbewerb'])

# Define conditions
verk = r'(?i)verkehrsw|mobilitätsw|verkehrsverl'
digi = r'(?i)digi'
nach = r'(?i)nachha|sustaina'
oeko = r'(?i)(?=(öko|umwelt|natur))(?!(ökonomi|naturgemäß|natürlich))'
spät = r'(?i)verpät|pünktl'
ener = r'(?i)energie|strom|solar|sonnenen|winde|strom|kohle|gas|erneuerbare|astroms'
klim = r'(?i)klima|co2|kohlenstoffdioxid|climate|carbon|treibhausg'
ref = r'(?i)reform|restruktur'
obso = r'(?i)obsoleszenz|repar|schaden|instandh|störung|instandse|stillstand|baustell|sani|renovi|mainten|wiederaufb|ausbess|veraltet'
wettbew = r'(?i)wettbe|konkurr|marktfü'

# Define masterframe
def analyse_text(file_path): 
    '''
    This code is used to apply regex search strings to the texts. It has the key operator `text`, which is why additional functions are called in here.
    The functions prepares to iterate through all PDF files in a given folder and pulls out matching word groups and their counts.
    '''
    with open(file_path, 'r') as f:
        text = textract.process(file_path)
        text = text.decode()
                                  
        # use regex calls for each file
        result_v = re.findall(verk, text, re.MULTILINE) # Verkehrswende
        results_v = len(result_v)
        
        result_d = re.findall(digi, text, re.MULTILINE) # Digitalisierung
        results_d = len(result_d) 

        result_n = re.findall(nach, text, re.MULTILINE) # Nachhaltigkeit
        results_n = len(result_n)                                   
                                  
        result_o = re.findall(oeko, text, re.MULTILINE) # Ökologie
        results_o = len(result_o)  

        result_s = re.findall(spät, text, re.MULTILINE) # IT
        results_s = len(result_s) 

        result_e = re.findall(ener, text, re.MULTILINE) # Energie
        results_e = len(result_e)                                   

        result_k = re.findall(klim, text, re.MULTILINE) # Klima
        results_k = len(result_k)        
        
        result_r = re.findall(ref, text, re.MULTILINE) # Reform
        results_r = len(result_r)
        
        result_ob = re.findall(obso, text, re.MULTILINE) # Obsoleszenz
        results_ob = len(result_ob)
        
        result_w = re.findall(wettbew, text, re.MULTILINE) # Wettbewerb
        results_w = len(result_w)
        
        # Count words in each pdf
        wordfreq = len(re.findall(r'\w+', text))
        
        # Pull into the master dataframe                          
        global master_df
        master_df_list = [{'Datei': fl,
                                      'Anzahl_Worte': wordfreq,
                                      'Verkehrswende': results_v,
                                      'Digital': results_d, 
                                      'Nachhaltig': results_n, 
                                      'Ökologie': results_o,
                                      'Energie': results_e,
                                      'Klima': results_k,
                                      'Reform': results_r,
                                      'Obsoleszenz': results_ob,
                                      'Verspätung': results_s,
                                      'Wettbewerb': results_w}] #, ignore_index=True]
        master_df = pd.concat([master_df, pd.DataFrame.from_records(master_df_list)])
        
        # print('Häufigkeit Wettbewerbsfähigkeit: ',results, 'Dokument: ', fl) 
        
# Start script
for file in os.listdir():
    '''
    The script iterates through a folder and saves/prints the results for the above regex search for each file.
    '''
    # Check whether file is in text format or not
    if file.endswith(".pdf"):
        file_path = f"{path}/{file}"
        fl = f"{file}"           
        # call read text file function
        analyse_text(file_path)

# show resulting dataframe and sort by year
master_df.sort_values(
    by='Datei',
    key=lambda x: np.argsort(index_natsorted(master_df['Datei'])),
    ascending=True
)

,Datei,Anzahl_Worte,Verkehrswende,Digital,Nachhaltig,Ökologie,Energie,Klima,Reform,Obsoleszenz,Verspätung,Wettbewerb
0,5-DB_GB_98.pdf,28922,4,1,6,9,38,0,29,18,14,30
0,6-DB_GB_99.pdf,33883,2,2,5,37,35,1,18,22,5,32
0,7-DB_GB_00.pdf,37469,2,3,8,41,54,8,28,51,4,53
0,8-DB_GB_01.pdf,38688,1,6,7,42,67,16,22,75,1,50
0,9-DB_GB_02.pdf,67287,6,4,23,49,101,16,27,95,3,93
0,10-DB_GB_03.pdf,67830,6,4,18,41,105,24,46,106,11,92
0,11-DB_GB_04.pdf,84650,9,6,25,52,138,17,34,88,13,105
0,12-DB_GB_05.pdf,82478,3,1,20,56,139,16,15,60,5,120
0,13-DB_GB_06.pdf,66966,4,1,17,42,133,15,9,55,5,70
0,14-DB_GB_07.pdf,90083,3,1,36,50,167,15,9,59,6,91


The original dataframe has the files as dataframe columns. In this case, however, the files can be equated with years, because they represent yearly reports. Hence simplification.

In [23]:
df=master_df.copy()
df=df.sort_values(
    by='Datei',
    key=lambda x: np.argsort(index_natsorted(master_df['Datei'])),
    ascending=True
)

df = df.reset_index(drop=True)
# df = df.drop(['index'], axis=1)
df.head() 

,Datei,Anzahl_Worte,Verkehrswende,Digital,Nachhaltig,Ökologie,Energie,Klima,Reform,Obsoleszenz,Verspätung,Wettbewerb
0,5-DB_GB_98.pdf,28922,4,1,6,9,38,0,29,18,14,30
1,6-DB_GB_99.pdf,33883,2,2,5,37,35,1,18,22,5,32
2,7-DB_GB_00.pdf,37469,2,3,8,41,54,8,28,51,4,53
3,8-DB_GB_01.pdf,38688,1,6,7,42,67,16,22,75,1,50
4,9-DB_GB_02.pdf,67287,6,4,23,49,101,16,27,95,3,93


In [24]:
# Rename file column
df = df.rename(columns={'Datei': 'Jahr'})

# Rename column content
df['Jahr'] = df['Jahr'].replace(['5-DB_GB_98.pdf',
                                 '6-DB_GB_99.pdf',
                                 '7-DB_GB_00.pdf',
                                 '8-DB_GB_01.pdf',
                                 '9-DB_GB_02.pdf',
                                 '10-DB_GB_03.pdf',
                                 '11-DB_GB_04.pdf',
                                 '12-DB_GB_05.pdf',
                                 '13-DB_GB_06.pdf',
                                 '14-DB_GB_07.pdf',
                                 '15-DB_GB_08.pdf',
                                 '16-DB_GB_09.pdf',
                                 '17-DB_GB_10.pdf',
                                 '18-DB_GB_11.pdf',
                                 '19-DB_GB_12.pdf',
                                 '20-DB_GB_13.pdf',
                                 '21-DB_IB_14.pdf',
                                 '22-DB_IB_15.pdf',
                                 '23-DB_IB_16.pdf',
                                 '24-DB_IB-17.pdf',
                                 '25-DB_IB_18.pdf',
                                 '26-DB_IB-19.pdf',
                                 '27-DB_IB20.pdf',
                                 ], 
                                 ['1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
                                 '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020'])     
# df.sort_values(by='Jahr', ascending=False)
df

,Jahr,Anzahl_Worte,Verkehrswende,Digital,Nachhaltig,Ökologie,Energie,Klima,Reform,Obsoleszenz,Verspätung,Wettbewerb
0,1998,28922,4,1,6,9,38,0,29,18,14,30
1,1999,33883,2,2,5,37,35,1,18,22,5,32
2,2000,37469,2,3,8,41,54,8,28,51,4,53
3,2001,38688,1,6,7,42,67,16,22,75,1,50
4,2002,67287,6,4,23,49,101,16,27,95,3,93
5,2003,67830,6,4,18,41,105,24,46,106,11,92
6,2004,84650,9,6,25,52,138,17,34,88,13,105
7,2005,82478,3,1,20,56,139,16,15,60,5,120
8,2006,66966,4,1,17,42,133,15,9,55,5,70
9,2007,90083,3,1,36,50,167,15,9,59,6,91


# Making Sense of the data distribution.

An important issue of the data is that the reports are of different length. We can clearly see the degrees to which a topic matters, but how much does it matter that a report gets longer? There is not a clear relationship, but looking at the data from different perspectives helps.

The first dataframe presents a bar char for each column where the chart indicates the relative size of each value compared to the hightest value in the column. Note that the bars only indicate the highest value for each column.

In [120]:
# Export to csv (first try from 2021)
df.to_csv('../6- Betriebsstörungen und SFB Siegen/Forschung/Dokumente/Publikationen DB/pdfs/frequency_count.csv')

,Jahr,Anzahl_Worte,Digitalisierung,Nachhaltigkeit,Ökologie,Reform,Ressourcen,Umwelt,Wettbewerb
0,1998,28922,1,6,1,27,2,7,29
1,1999,33883,2,5,11,14,8,26,27
2,2000,37469,3,8,8,22,5,31,51
3,2001,38688,6,7,9,17,6,30,45
4,2002,67287,4,23,10,16,9,33,88
5,2003,67830,4,17,6,40,5,27,88
6,2004,84650,6,25,8,24,8,40,101
7,2005,82478,1,20,5,9,12,46,110
8,2006,66966,1,17,7,6,9,32,65
9,2007,90083,1,36,9,7,8,38,86


In [25]:
# Export to csv (update from 2022)
df.to_csv('../Dokumente/Publikationen DB/pdfs/frequency_count_update_2022_.csv')

The next dataframe calculates the values of a column relative to the length of the entire PDF.

In [ ]:
# Original test from 2021
# Note that the values are not true to counting but based on rounding
df_relative = df.copy()
df_relative['Digitalisierung'] = ((100000 * df_relative['Digitalisierung']) / df_relative['Anzahl_Worte']).astype(int)
df_relative['Nachhaltigkeit'] = ((100000 * df_relative['Nachhaltigkeit']) / df_relative['Anzahl_Worte']).astype(int)
#df_relative['Ökologie'] = ((100000 * df_relative['Ökologie']) / df_relative['Anzahl_Worte']).astype(int)
df_relative['Reform'] = ((100000 * df_relative['Reform']) / df_relative['Anzahl_Worte']).astype(int)
df_relative['Ressourcen'] = ((100000 * df_relative['Ressourcen']) / df_relative['Anzahl_Worte']).astype(int)
df_relative['Umwelt'] = ((100000 * df_relative['Umwelt']) / df_relative['Anzahl_Worte']).astype(int)
df_relative['Wettbewerb'] = ((100000 * df_relative['Wettbewerb']) / df_relative['Anzahl_Worte']).astype(int)
df_relative

KeyError: 'Ressourcen'

In [27]:
# Update from 2022
# Note that the values are not true to counting but based on rounding
df_relative = df.copy()
df_relative['Verkehrswende'] = ((100000 * df_relative['Verkehrswende']) / df_relative['Anzahl_Worte']).astype(int)
df_relative['Digital'] = ((100000 * df_relative['Digital']) / df_relative['Anzahl_Worte']).astype(int)
df_relative['Nachhaltig'] = ((100000 * df_relative['Nachhaltig']) / df_relative['Anzahl_Worte']).astype(int)
df_relative['Ökologie'] = ((100000 * df_relative['Ökologie']) / df_relative['Anzahl_Worte']).astype(int)
df_relative['Verspätung'] = ((100000 * df_relative['Verspätung']) / df_relative['Anzahl_Worte']).astype(int)
df_relative['Energie'] = ((100000 * df_relative['Energie']) / df_relative['Anzahl_Worte']).astype(int)
df_relative['Klima'] = ((100000 * df_relative['Klima']) / df_relative['Anzahl_Worte']).astype(int)
df_relative['Reform'] = ((100000 * df_relative['Reform']) / df_relative['Anzahl_Worte']).astype(int)
df_relative['Obsoleszenz'] = ((100000 * df_relative['Obsoleszenz']) / df_relative['Anzahl_Worte']).astype(int)
df_relative['Wettbewerb'] = ((100000 * df_relative['Wettbewerb']) / df_relative['Anzahl_Worte']).astype(int)
df_relative

,Jahr,Anzahl_Worte,Verkehrswende,Digital,Nachhaltig,Ökologie,Energie,Klima,Reform,Obsoleszenz,Verspätung,Wettbewerb
0,1998,28922,13,3,20,31,131,0,100,62,48,103
1,1999,33883,5,5,14,109,103,2,53,64,14,94
2,2000,37469,5,8,21,109,144,21,74,136,10,141
3,2001,38688,2,15,18,108,173,41,56,193,2,129
4,2002,67287,8,5,34,72,150,23,40,141,4,138
5,2003,67830,8,5,26,60,154,35,67,156,16,135
6,2004,84650,10,7,29,61,163,20,40,103,15,124
7,2005,82478,3,1,24,67,168,19,18,72,6,145
8,2006,66966,5,1,25,62,198,22,13,82,7,104
9,2007,90083,3,1,39,55,185,16,9,65,6,101


In [28]:
df_relative.to_csv('../Dokumente/Publikationen DB/pdfs/frequency_count_relative_2022_.csv') # saved with Ökologie

In [153]:
df_relative.style.bar(subset=['Anzahl_Worte', 'Digitalisierung', 'Nachhaltigkeit', 'Ökologie', 'Reform', 'Ressourcen', 'Umwelt', 'Wettbewerb'], color=['#a1d99b'])

,Jahr,Anzahl_Worte,Digitalisierung,Nachhaltigkeit,Ökologie,Reform,Ressourcen,Umwelt,Wettbewerb
0,1998,28922,3,20,3,93,6,24,100
1,1999,33883,5,14,32,41,23,76,79
2,2000,37469,8,21,21,58,13,82,136
3,2001,38688,15,18,23,43,15,77,116
4,2002,67287,5,34,14,23,13,49,130
5,2003,67830,5,25,8,58,7,39,129
6,2004,84650,7,29,9,28,9,47,119
7,2005,82478,1,24,6,10,14,55,133
8,2006,66966,1,25,10,8,13,47,97
9,2007,90083,1,39,9,7,8,42,95


The conclusion: Digital and sustinability matters have become significantly more important during the past decade, with the programme Starke Schiene in 2019 really making a difference. There is also a notable decline of matters of competition, at least in its traditional phrasing, although it remainds relevant. Besides, note that ecology (framed as "Umwelt") used to be a relevant topic during the late 1990s and early 2000s, but then vanished for roughly a decade and then got "overtaken" by ecology and then sustainability in particular. "Ressources" are rather ambigious and are not very helpful, although it can become a relatively important term in certain sectors.

# Count words: Single code for testing

In [ ]:
# define search string
# wettbew = re.findall(r'\s(?i)wettbew', text, re.MULTILINE) # Sucht nach allen strings "wettbe", case insensitive, mit whitespace davor

# Testet conditions
verk = r'(?i)verkehrsw|mobilitätsw|verkehrsverl
digi = r'(?i)digit'
nach = r'(?i)nachha|sustaina'
oeko = r'(?i)öko' # (?i)öko[^n] war nur bedingt hilfreich
ener = r'(?i)energie|strom|kohle|gas|erneuerbare'
klim = r'(?i)klima|co2|kohlenstoffdioxid|climate|carbon|treibhausg'
ref = r'(?i)reform'
obso = r'(?i)obsoleszenz|repar|schaden|instandh|stillstand|baustell|zuverlässig'
wettbew = r'(?i)wettbe|konkurr|rentab|gewinn|profit|wirtschaftlich'


# Nach Tests auf Grund niedriger Statistiken wenig aussagekräftig: standard, investi, ressourcen, IT-

In [15]:
# Parsing with textract
text_test = textract.process("../Dokumente/Publikationen DB/pdfs/22-DB_IB_15.pdf")
text_test = text_test.decode() # Transform bytes to string

# Search for words
test = re.findall(r'(?i)verpät|pünktl', text_test, re.MULTILINE) # Sucht nach allen strings, case insensitive, mit whitespace davor

# Check results
print(len(test))

90
